In [29]:
import os
import glob
import warnings
import random
import json
import logging
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import zipfile
from typing import Tuple, List, Dict, Any, Optional

from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import mlflow
import mlflow.pytorch
from mlflow.tracking import MlflowClient

warnings.filterwarnings('ignore')

In [30]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [31]:
class CNNConfig:
    #ini masih single frame classification, next stepnya lstm + sequential classification
    
    IMAGE_SIZE = (224, 224)  
    NUM_CLASSES = 2  
    

    MODEL_NAME = "efficientnet_b7"
    CNN_FEATURE_SIZE = 2560  
    DROPOUT = 0.4
    
    K_FOLDS = 5
    FINAL_TEST_SIZE = 0.15
    BATCH_SIZE = 8  
    EPOCHS = 30
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4
    PATIENCE = 10
    MIN_DELTA = 0.001
    

    ROTATION_RANGE = 15
    BRIGHTNESS_RANGE = 0.2
    CONTRAST_RANGE = 0.2
    
    #ini gw namain foldernya event, ubah aja sesuai penamaan
    DATA_DIR = "event/The Dataset"
    MODEL_DIR = "models/phase1_cnn"
    RESULTS_DIR = "results/phase1_cnn"
    LOGS_DIR = "logs"
    
    MLFLOW_EXPERIMENT_NAME = "car_exit_detection_phase1_cnn"
    MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"  
    
    def __init__(self):
        self.create_directories()
        self.setup_mlflow()
    
    def create_directories(self):
        directories = [
            self.MODEL_DIR, 
            self.RESULTS_DIR, 
            self.LOGS_DIR,
            "mlruns"
        ]
        
        for dir_path in directories:
            os.makedirs(dir_path, exist_ok=True)
            logger.info(f"Created directory: {dir_path}")
    
    def setup_mlflow(self):
        
        mlflow.set_tracking_uri(self.MLFLOW_TRACKING_URI)
        
        try:

            mlflow.set_experiment(self.MLFLOW_EXPERIMENT_NAME)

            experiment = mlflow.get_experiment_by_name(self.MLFLOW_EXPERIMENT_NAME)
            if experiment is None:
                experiment_id = mlflow.create_experiment(self.MLFLOW_EXPERIMENT_NAME)
                logger.info(f"Created MLFlow experiment: {self.MLFLOW_EXPERIMENT_NAME}")
            else:
                experiment_id = experiment.experiment_id
                logger.info(f"Using existing MLFlow experiment: {self.MLFLOW_EXPERIMENT_NAME}")
            
            
        except Exception as e:
            logger.error(f"MLFlow setup failed: {e}")
            raise

In [32]:
class CarExitCNNDataset(Dataset):
    """

    masih single frame classfication
    
    loads individual images from sequence folders and treats
    each image as an independent sample for CNN training.

    """
    
    def __init__(self, image_paths: List[str], labels: List[int], transform=None):
      
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
        logger.info(f"Dataset initialized with {len(image_paths)} images")
        logger.info(f"Class distribution: {dict(zip(*np.unique(labels, return_counts=True)))}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        try:
            image_path = self.image_paths[idx]
            label = self.labels[idx]
            
            image = Image.open(image_path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
            
            return image, torch.tensor(label, dtype=torch.long)
            
        except Exception as e:
            logger.warning(f"Error loading image {self.image_paths[idx]}: {e}")
            return None

In [33]:
class CNNCarExitClassifier(nn.Module):

    
    def __init__(self, num_classes: int = 2, dropout: float = 0.4, cnn_feature_size: int = 2560):
   
        super(CNNCarExitClassifier, self).__init__()
        
        self.num_classes = num_classes
        self.dropout = dropout
        

        self.backbone = efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1)
        

        self.backbone.classifier = nn.Identity()
        
        self._freeze_backbone()
        
        self.classifier = nn.Sequential(
            nn.Linear(cnn_feature_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout * 0.5),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout * 0.25),
            
            nn.Linear(256, num_classes)
        )
        
        self._initialize_classifier_weights()
        
        logger.info(f"CNN Classifier initialized with {self._count_parameters():,} parameters")
    
    def _freeze_backbone(self):

        for param in self.backbone.parameters():
            param.requires_grad = False
        logger.info("Backbone frozen for stable training")
    
    def unfreeze_backbone(self, unfreeze_layers: int = -1):
        
        if unfreeze_layers == -1:
            for param in self.backbone.parameters():
                param.requires_grad = True
            logger.info("All backbone layers unfrozen for fine-tuning")
        else:
            layers = list(self.backbone.features.children())
            for layer in layers[-unfreeze_layers:]:
                for param in layer.parameters():
                    param.requires_grad = True
            logger.info(f"Last {unfreeze_layers} backbone layers unfrozen for fine-tuning")
    
    def _initialize_classifier_weights(self):

        for module in self.classifier.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.BatchNorm1d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
    
    def _count_parameters(self) -> int:

        return sum(p.numel() for p in self.parameters() if p.requires_grad)
    
    def forward(self, x):
      
        features = self.backbone(x)
        
        logits = self.classifier(features)
        
        return logits

In [34]:
class CNNTrainer:

    def __init__(self, config: CNNConfig, device: torch.device):
        self.config = config
        self.device = device
        self.client = MlflowClient()
        
        logger.info(f"Trainer initialized on device: {device}")
    
    def create_transforms(self) -> Tuple[transforms.Compose, transforms.Compose]:
    
        train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomResizedCrop(self.config.IMAGE_SIZE, scale=(0.85, 1.0)),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomRotation(degrees=self.config.ROTATION_RANGE),
            transforms.ColorJitter(
                brightness=self.config.BRIGHTNESS_RANGE,
                contrast=self.config.CONTRAST_RANGE,
                saturation=0.1,
                hue=0.05
            ),
            transforms.RandomApply([
                transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))
            ], p=0.1),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
            transforms.RandomErasing(p=0.1, scale=(0.02, 0.1))
        ])
        
        val_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(self.config.IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
        
        return train_transform, val_transform
    
    def train_fold(self, model: nn.Module, train_loader: DataLoader, 
                   val_loader: DataLoader, fold_num: int) -> Tuple[nn.Module, Dict, float]:

        optimizer = AdamW(
            model.parameters(),
            lr=self.config.LEARNING_RATE,
            weight_decay=self.config.WEIGHT_DECAY
        )
        
        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.config.LEARNING_RATE * 3,
            epochs=self.config.EPOCHS,
            steps_per_epoch=len(train_loader),
            pct_start=0.3
        )
        
        criterion = nn.CrossEntropyLoss()
        
        history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': []
        }
        
        best_val_acc = 0.0
        patience_counter = 0
        best_model_state = None
        
        logger.info(f"Training Fold {fold_num} - {len(train_loader.dataset)} samples")
        
        for epoch in range(self.config.EPOCHS):
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            
            pbar = tqdm(train_loader, desc=f'Fold {fold_num} Epoch {epoch+1}', leave=False)
            for images, labels in pbar:
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                scheduler.step()
                
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
                
                pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100.*train_correct/train_total:.1f}%'
                })
            
            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
            
            epoch_train_loss = train_loss / len(train_loader)
            epoch_train_acc = train_correct / train_total
            epoch_val_loss = val_loss / len(val_loader)
            epoch_val_acc = val_correct / val_total
            
            history['train_loss'].append(epoch_train_loss)
            history['train_acc'].append(epoch_train_acc)
            history['val_loss'].append(epoch_val_loss)
            history['val_acc'].append(epoch_val_acc)
            
            mlflow.log_metrics({
                f'fold_{fold_num}_train_loss': epoch_train_loss,
                f'fold_{fold_num}_train_acc': epoch_train_acc,
                f'fold_{fold_num}_val_loss': epoch_val_loss,
                f'fold_{fold_num}_val_acc': epoch_val_acc
            }, step=epoch)
            
            logger.info(f"Epoch {epoch+1:2d}/{self.config.EPOCHS} "
                       f"Train: {epoch_train_loss:.4f}/{epoch_train_acc:.4f} "
                       f"Val: {epoch_val_loss:.4f}/{epoch_val_acc:.4f}")
            

            if epoch_val_acc > best_val_acc + self.config.MIN_DELTA:
                best_val_acc = epoch_val_acc
                patience_counter = 0
                best_model_state = model.state_dict().copy()
                logger.info(f"New best model - Val Acc: {epoch_val_acc:.4f}")
            else:
                patience_counter += 1
    
            if patience_counter >= self.config.PATIENCE:
                logger.info(f"Early stopping at epoch {epoch+1}")
                break
        
        if best_model_state:
            model.load_state_dict(best_model_state)
        
        return model, history, best_val_acc
    
    def cross_validate(self, dataset: CarExitCNNDataset) -> Tuple[List[Dict], Dict, nn.Module]:

        with mlflow.start_run(run_name=f"cnn_cv_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
            
            mlflow.log_params({
                'model_type': 'CNN_EfficientNet_B7',
                'k_folds': self.config.K_FOLDS,
                'batch_size': self.config.BATCH_SIZE,
                'epochs': self.config.EPOCHS,
                'learning_rate': self.config.LEARNING_RATE,
                'dropout': self.config.DROPOUT,
                'image_size': str(self.config.IMAGE_SIZE)
            })
            
            labels = np.array(dataset.labels)
            skf = StratifiedKFold(n_splits=self.config.K_FOLDS, shuffle=True, random_state=42)
            
            fold_results = []
            all_models = []
            
            for fold, (train_idx, val_idx) in enumerate(skf.split(range(len(dataset)), labels)):
                logger.info(f"\nFOLD {fold + 1}/{self.config.K_FOLDS}")
                
                train_sampler = SubsetRandomSampler(train_idx)
                val_sampler = SubsetRandomSampler(val_idx)
            
                train_loader = DataLoader(
                    dataset,
                    batch_size=self.config.BATCH_SIZE,
                    sampler=train_sampler,
                    num_workers=0,
                    pin_memory=True
                )
                
                val_loader = DataLoader(
                    dataset,
                    batch_size=self.config.BATCH_SIZE,
                    sampler=val_sampler,
                    num_workers=0,
                    pin_memory=True
                )
                
                logger.info(f"Training samples: {len(train_idx)}, Validation samples: {len(val_idx)}")
                
                model = CNNCarExitClassifier(
                    num_classes=self.config.NUM_CLASSES,
                    dropout=self.config.DROPOUT,
                    cnn_feature_size=self.config.CNN_FEATURE_SIZE
                ).to(self.device)
                
                model, history, best_val_acc = self.train_fold(model, train_loader, val_loader, fold + 1)
                
                fold_result = {
                    'fold': fold + 1,
                    'best_val_acc': best_val_acc,
                    'final_train_acc': history['train_acc'][-1],
                    'final_val_acc': history['val_acc'][-1],
                    'history': history
                }
                
                fold_results.append(fold_result)
                all_models.append(model)
                
                fold_model_path = os.path.join(self.config.MODEL_DIR, f'fold_{fold+1}_model.pth')
                torch.save({
                    'fold': fold + 1,
                    'model_state_dict': model.state_dict(),
                    'best_val_acc': best_val_acc,
                    'fold_result': fold_result
                }, fold_model_path)
                
                mlflow.log_artifact(fold_model_path, f"models/fold_{fold+1}")
                
                logger.info(f"✅ Fold {fold+1} completed - Best Val Acc: {best_val_acc:.4f}")
            
            cv_stats = self._calculate_cv_statistics(fold_results)
            
            # Log CV statistics
            mlflow.log_metrics({
                'cv_mean_val_acc': cv_stats['mean_val_acc'],
                'cv_std_val_acc': cv_stats['std_val_acc'],
                'cv_mean_train_acc': cv_stats['mean_train_acc'],
                'cv_std_train_acc': cv_stats['std_train_acc']
            })
            
            # save best model
            best_fold_idx = np.argmax([r['best_val_acc'] for r in fold_results])
            best_model = all_models[best_fold_idx]
            
            best_model_path = os.path.join(self.config.MODEL_DIR, 'best_cnn_model.pth')
            torch.save({
                'model_state_dict': best_model.state_dict(),
                'cv_stats': cv_stats,
                'best_fold': best_fold_idx + 1,
                'config': self.config.__dict__
            }, best_model_path)
            
            # Log best model
            mlflow.pytorch.log_model(best_model, "best_model")
            mlflow.log_artifact(best_model_path, "models")
            
            return fold_results, cv_stats, best_model

    def _calculate_cv_statistics(self, fold_results: List[Dict]) -> Dict:

        val_accs = [r['best_val_acc'] for r in fold_results]
        train_accs = [r['final_train_acc'] for r in fold_results]
        
        cv_stats = {
            'mean_val_acc': np.mean(val_accs),
            'std_val_acc': np.std(val_accs),
            'mean_train_acc': np.mean(train_accs),
            'std_train_acc': np.std(train_accs),
            'fold_results': fold_results
        }
        
        logger.info(f"\nCROSS-VALIDATION RESULTS")
        logger.info(f"Mean Validation Accuracy: {cv_stats['mean_val_acc']:.4f} ± {cv_stats['std_val_acc']:.4f}")
        logger.info(f"Mean Training Accuracy: {cv_stats['mean_train_acc']:.4f} ± {cv_stats['std_train_acc']:.4f}")
        
        return cv_stats


In [35]:
def load_image_data(data_dir: str) -> Tuple[List[str], List[int]]:
    image_paths = []
    labels = []
    
    for class_label in [0, 1]:
        class_dir = os.path.join(data_dir, str(class_label))
        
        if not os.path.exists(class_dir):
            logger.warning(f"Class directory {class_dir} not found")
            continue
        
        sequence_folders = [d for d in os.listdir(class_dir) 
                           if os.path.isdir(os.path.join(class_dir, d))]
        
        for seq_folder in sequence_folders:
            seq_path = os.path.join(class_dir, seq_folder)
            
            extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
            sequence_images = []
            for ext in extensions:
                sequence_images.extend(glob.glob(os.path.join(seq_path, ext)))
            
            for img_path in sequence_images:
                image_paths.append(img_path)
                labels.append(class_label)
    
    logger.info(f"Dataset loaded:")
    logger.info(f"Total images: {len(image_paths)}")
    logger.info(f"Class 0 (Normal): {labels.count(0)} images")
    logger.info(f"Class 1 (Person Exiting): {labels.count(1)} images")
    
    if len(image_paths) == 0:
        raise ValueError("No images found.")
    
    return image_paths, labels




In [36]:
def visualize_results(cv_stats: Dict, config: CNNConfig):

    fold_results = cv_stats['fold_results']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('CNN Phase 1 - Cross Validation Results', fontsize=16, fontweight='bold')
    
    folds = [r['fold'] for r in fold_results]
    train_accs = [r['final_train_acc'] for r in fold_results]
    val_accs = [r['best_val_acc'] for r in fold_results]
    
    x = np.arange(len(folds))
    width = 0.35
    
    axes[0, 0].bar(x - width/2, train_accs, width, label='Training', alpha=0.8, color='skyblue')
    axes[0, 0].bar(x + width/2, val_accs, width, label='Validation', alpha=0.8, color='lightcoral')
    axes[0, 0].set_title('Accuracy by Fold')
    axes[0, 0].set_xlabel('Fold')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].set_xticks(x)
    axes[0, 0].set_xticklabels(folds)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    best_fold_idx = np.argmax(val_accs)
    best_history = fold_results[best_fold_idx]['history']
    
    epochs = range(1, len(best_history['train_acc']) + 1)
    axes[0, 1].plot(epochs, best_history['train_acc'], 'b-', label='Training Accuracy', linewidth=2)
    axes[0, 1].plot(epochs, best_history['val_acc'], 'r-', label='Validation Accuracy', linewidth=2)
    axes[0, 1].set_title(f'Best Fold ({best_fold_idx+1}) - Training History')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    mean_val = cv_stats['mean_val_acc']
    std_val = cv_stats['std_val_acc']
    
    axes[1, 0].errorbar([1], [mean_val], yerr=[std_val], 
                       fmt='o', markersize=15, capsize=10, capthick=3, color='green')
    axes[1, 0].set_xlim([0.5, 1.5])
    axes[1, 0].set_ylim([max(0, mean_val - 2*std_val), min(1, mean_val + 2*std_val)])
    axes[1, 0].set_title(f'CV Mean ± Std\n{mean_val:.4f} ± {std_val:.4f}')
    axes[1, 0].set_xticks([])
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].plot(epochs, best_history['train_loss'], 'b-', label='Training Loss', linewidth=2)
    axes[1, 1].plot(epochs, best_history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    axes[1, 1].set_title(f'Best Fold ({best_fold_idx+1}) - Loss History')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    plot_path = os.path.join(config.RESULTS_DIR, 'cv_results_visualization.png')
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    mlflow.log_artifact(plot_path)
    return plot_path
    

In [37]:
def predict_single_image(model_path: str, image_path: str, config: CNNConfig, 
                        return_probabilities: bool = True) -> Dict[str, Any]:
    
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        model = CNNCarExitClassifier(
            num_classes=config.NUM_CLASSES,
            dropout=config.DROPOUT
        )
        
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        model.eval()
        
        _, val_transform = CNNTrainer(config, device).create_transforms()
        
        image = Image.open(image_path).convert('RGB')
        image_tensor = val_transform(image).unsqueeze(0).to(device)
        

        with torch.no_grad():
            outputs = model(image_tensor)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            prediction = predicted.cpu().item()
            confidence = probabilities[0, prediction].cpu().item()
            
            result = {
                'prediction': prediction,
                'class_name': 'person_exiting' if prediction == 1 else 'normal_parking',
                'confidence': confidence,
                'image_path': image_path
            }
            
            if return_probabilities:
                result['probabilities'] = {
                    'normal_parking': probabilities[0, 0].cpu().item(),
                    'person_exiting': probabilities[0, 1].cpu().item()
                }
            
            logger.info(f"Prediction: {result['class_name']} (confidence: {confidence:.4f})")
            
            return result
            
    except Exception as e:
        logger.error(f"Prediction failed: {e}")
        return {'error': f'Prediction failed: {str(e)}'}

In [38]:
def evaluate_test_set(model: nn.Module, test_loader: DataLoader, device: torch.device, 
                     config: CNNConfig) -> Dict[str, float]:
 
    model.eval()
    test_correct = 0
    test_total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating test set"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
  
    test_accuracy = test_correct / test_total
    test_f1 = f1_score(all_labels, all_predictions, average='weighted')
    test_precision = precision_score(all_labels, all_predictions, average='weighted')
    test_recall = recall_score(all_labels, all_predictions, average='weighted')
    

    test_f1_per_class = f1_score(all_labels, all_predictions, average=None)
    test_precision_per_class = precision_score(all_labels, all_predictions, average=None)
    test_recall_per_class = recall_score(all_labels, all_predictions, average=None)
    
    metrics = {
        'accuracy': test_accuracy,
        'f1_weighted': test_f1,
        'precision_weighted': test_precision,
        'recall_weighted': test_recall,
        'f1_normal': test_f1_per_class[0],
        'f1_person_exiting': test_f1_per_class[1],
        'precision_normal': test_precision_per_class[0],
        'precision_person_exiting': test_precision_per_class[1],
        'recall_normal': test_recall_per_class[0],
        'recall_person_exiting': test_recall_per_class[1],
        'test_samples': test_total
    }
    

    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=['Normal Parking', 'Person Exiting'],
               yticklabels=['Normal Parking', 'Person Exiting'])
    plt.title('Test Set Confusion Matrix', fontweight='bold', fontsize=14)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
  
    cm_path = os.path.join(config.RESULTS_DIR, 'test_confusion_matrix.png')
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    plt.show()
    

    mlflow.log_artifact(cm_path)
    

    class_names = ['Normal Parking', 'Person Exiting']
    report = classification_report(all_labels, all_predictions, target_names=class_names)
    logger.info(f"\nTEST SET EVALUATION:")
    logger.info(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    logger.info(f"Test F1-Score (weighted): {test_f1:.4f}")
    logger.info(f"Test Precision (weighted): {test_precision:.4f}")
    logger.info(f"Test Recall (weighted): {test_recall:.4f}")
    logger.info(f"\nDetailed Classification Report:\n{report}")
    
    return metrics

In [39]:
def main():

    try:
 
        config = CNNConfig()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Device: {device}")
        
        if torch.cuda.is_available():
            logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        
        if os.path.exists("dataset_parking.zip") and not os.path.exists(config.DATA_DIR):
            os.makedirs("event", exist_ok=True)
            with zipfile.ZipFile("dataset_parking.zip", 'r') as zip_ref:
                zip_ref.extractall("event")
        
        # Load data
        image_paths, labels = load_image_data(config.DATA_DIR)
        
        # Check class balance
        class_counts = {0: labels.count(0), 1: labels.count(1)}
        logger.info(f"Class distribution: {class_counts}")
        
        if config.FINAL_TEST_SIZE > 0:
            train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
                image_paths, labels,
                test_size=config.FINAL_TEST_SIZE,
                stratify=labels,
                random_state=42
            )
            logger.info(f"Data split:")
            logger.info(f"Cross-validation set: {len(train_val_paths)} images")
            logger.info(f"Final test set: {len(test_paths)} images")
        else:
            train_val_paths, train_val_labels = image_paths, labels
            test_paths, test_labels = [], []
            logger.info(f"Using all {len(image_paths)} images for cross-validation")
        
        # Create datasets
        trainer = CNNTrainer(config, device)
        train_transform, val_transform = trainer.create_transforms()
        
        # Cross-validation dataset 
        cv_dataset = CarExitCNNDataset(train_val_paths, train_val_labels, transform=train_transform)
        
        # Perform cross-validation
        fold_results, cv_stats, best_model = trainer.cross_validate(cv_dataset)
        
        # Visualize results
        visualize_results(cv_stats, config)
        
        # Evaluate on test set if available
        test_metrics = None
        if test_paths:
            test_dataset = CarExitCNNDataset(test_paths, test_labels, transform=val_transform)
            test_loader = DataLoader(
                test_dataset,
                batch_size=config.BATCH_SIZE,
                shuffle=False,
                num_workers=0,
                pin_memory=True
            )
            
            test_metrics = evaluate_test_set(best_model, test_loader, device, config)
            
            mlflow.log_metrics({
                'test_accuracy': test_metrics['accuracy'],
                'test_f1_weighted': test_metrics['f1_weighted'],
                'test_precision_weighted': test_metrics['precision_weighted'],
                'test_recall_weighted': test_metrics['recall_weighted']
            })
        
        results_summary = {
            'timestamp': datetime.now().isoformat(),
            'phase': 'Phase 1 - CNN Implementation',
            'model_architecture': 'EfficientNet-B7 + Custom Classifier',
            'config': {
                'k_folds': config.K_FOLDS,
                'model_name': config.MODEL_NAME,
                'batch_size': config.BATCH_SIZE,
                'epochs': config.EPOCHS,
                'learning_rate': config.LEARNING_RATE,
                'dropout': config.DROPOUT,
                'image_size': config.IMAGE_SIZE
            },
            'data_info': {
                'total_images': len(image_paths),
                'train_val_images': len(train_val_paths),
                'test_images': len(test_paths) if test_paths else 0,
                'class_distribution': class_counts
            },
            'cross_validation': {
                'mean_val_accuracy': cv_stats['mean_val_acc'],
                'std_val_accuracy': cv_stats['std_val_acc'],
                'mean_train_accuracy': cv_stats['mean_train_acc'],
                'std_train_accuracy': cv_stats['std_train_acc'],
                'fold_results': [
                    {
                        'fold': r['fold'],
                        'best_val_acc': r['best_val_acc'],
                        'final_train_acc': r['final_train_acc']
                    } for r in fold_results
                ]
            }
        }
        
        if test_metrics:
            results_summary['final_test'] = test_metrics
        
        results_path = os.path.join(config.RESULTS_DIR, 'phase1_cnn_results.json')
        with open(results_path, 'w') as f:
            json.dump(results_summary, f, indent=2)
        
        mlflow.log_artifact(results_path)
        
        logger.info(f"Cross-Validation Accuracy: {cv_stats['mean_val_acc']:.4f} ± {cv_stats['std_val_acc']:.4f}")
        
        if test_metrics:
            logger.info(f"Final Test Accuracy: {test_metrics['accuracy']:.4f}")
        
        logger.info(f"\nSAVED FILES:")
        logger.info(f"   Best model: {config.MODEL_DIR}/best_cnn_model.pth")
        logger.info(f"   Individual folds: {config.MODEL_DIR}/fold_*_model.pth")
        logger.info(f"   Results summary: {results_path}")
        logger.info(f"   Visualizations: {config.RESULTS_DIR}/")
        logger.info(f"   MLFlow tracking: {config.MLFLOW_TRACKING_URI}")
        
        return best_model, cv_stats, results_summary, test_metrics
        
    except Exception as e:
        logger.error(f"Training pipeline failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None, None





In [40]:
def run_prediction_test():
 
    config = CNNConfig()

    try:
        # Single image prediction 
        # next-step mulai sequential 
        result = predict_single_image(
            model_path=os.path.join(config.MODEL_DIR, 'best_cnn_model.pth'),
            image_path='frame_015.jpg',
            config=config
        )
        
        if 'error' not in result:
            print(f"Prediction: {result['class_name']}")
            print(f"Confidence: {result['confidence']:.4f}")
            if 'probabilities' in result:
                for class_name, prob in result['probabilities'].items():
                    print(f"  {class_name}: {prob:.4f}")
        else:
            print(f"Error: {result['error']}")
            
    except Exception as e:
        print(f"Prediction example failed: {e}")




In [41]:
if __name__ == "__main__":

    model, cv_stats, results, test_metrics = main()
    
    if model is not None:
        run_prediction_test()

    else:
        logger.error("\nPhase 1 CNN implementation failed")


Fold 1 Epoch 1:   0%|          | 0/11 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'CarExitCNNDataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.12/3.12.11/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent